In [1]:
import os
import sys
import pandas as pd
import numpy as np
import glob
import time
import scipy
from scipy.sparse import csr_matrix
import anndata as an
import scanpy as sc
import pyBigWig

In [2]:
anndata_path = "/scratch/indikar_root/indikar1/shared_data/higher_order/anndata/population_mESC_1000000_raw.h5ad"

start_time = time.time()  # Record the start time
adata = sc.read_h5ad(anndata_path)
end_time = time.time()  # Record the end time
print(f"Time taken to read the file: {end_time - start_time:.2f} seconds")
sc.logging.print_memory_usage()
adata

Time taken to read the file: 23.76 seconds
Memory usage: current 1.38 GB, difference +1.38 GB


AnnData object with n_obs × n_vars = 2579 × 2756467
    obs: 'bin_index', 'chrom', 'bin_start', 'bin_end', 'bin', 'chrom_bin', 'n_reads', 'n_genes', 'n_tfs', 'n_pt_genes', 'total_gene_bp', 'genes', 'total_len_bp'
    var: 'read_index', 'mean_mapq', 'median_mapq', 'n_chromosomes', 'order', 'n_bins', 'read_length_bp'
    uns: 'base_resolution', 'chrom_sizes', 'genes', 'intervals'
    layers: 'H'

In [6]:
adata.obs.head()

,bin_index,chrom,bin_start,bin_end,bin,chrom_bin,n_reads,n_genes,n_tfs,n_pt_genes,total_gene_bp,genes,total_len_bp
bin_name,,,,,,,,,,,,,
chr1:153,0,1,153000000,154000000,153,153,5728,31,0,13.0,616889.0,Gm8818;Shcbp1l;Gm25588;Gm37669;Rgs8;5530400K19...,616889
chr1:66,1,1,66000000,67000000,66,66,5065,27,0,6.0,710064.0,Gm15659;Gm8812;Map2;Gm24729;Gm25411;Unc80;Rpe;...,710064
chr1:23,2,1,23000000,24000000,23,23,4774,15,0,4.0,413106.0,Gm24176;Mir30a;Gm27028;Mir30c-2;Gm29107;Gm7784...,413106
chr1:188,3,1,188000000,189000000,188,188,5367,8,0,3.0,797832.0,Ush2a;Gm25269;Gm37178;Gm25095;Kctd3;Gm18698;Kc...,797832
chr1:97,4,1,97000000,98000000,97,97,4444,14,0,5.0,779945.0,Gm7133;Gm53038;Gin1;Gm56241;Gm24465;Gm7135;Slc...,779945


In [3]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
adata.obs['bin_end'].min()

In [ ]:
feature_path = "/scratch/indikar_root/indikar1/shared_data/higher_order/features/ATACSeq_1.bw"
bw = pyBigWig.open(feature_path)  
basename = os.path.basename(feature_path).replace(".bw", "")
print(basename)

feature_map = {}
start_time = time.time()  # Record the start time
row_count = 0  # Initialize a counter

for idx, row in adata.obs.iterrows():
    chrom = row['chrom']
    start = row['bin_start']
    end = row['bin_end']
    try:
        value = bw.stats(f"chr{chrom}", start, end, type='mean')[0]
        feature_map[idx] = value
    except:
        print(f"ERROR: {idx}")
        feature_map[idx] = 0.0

    # Print periodic status updates
    row_count += 1
    if row_count % 10000 == 0:  # Adjust the interval as needed
        elapsed_time = time.time() - start_time
        print(f"Processed {row_count} rows in {elapsed_time:.2f} seconds")

adata.obs[basename] = adata.obs.index.map(feature_map)
adata.obs[basename].head()

In [ ]:
adata.obs[['n_reads', 'ATACSeq_1']].corr()